In [36]:
from ultralytics import YOLO
from torch.ao.quantization import prepare, convert

# Load the YOLOv8 model
model_path = "yolov8s_best.pt"
model = YOLO(model_path).model  # Get the PyTorch model

# Put the model in evaluation mode
model.eval()

WARNING  yolov8s_best.pt appears to require 'numpy._core', which is not in Ultralytics requirements.
AutoInstall will run now for 'numpy._core' but this feature will be removed in the future.
Recommend fixes are to train a new model using the latest 'ultralytics' package or to run a command with an official Ultralytics model, i.e. 'yolo predict model=yolov8n.pt'


ModuleNotFoundError: No module named 'numpy._core'

In [29]:
import torch
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from pathlib import Path
from PIL import Image

# Custom Dataset for Calibration
class CalibrationDataset(Dataset):
    def __init__(self, image_dir, transform=None):
        self.image_paths = [str(p) for p in Path(image_dir).rglob("*.[jp][pn]*")]
        if len(self.image_paths) == 0:
            raise ValueError(f"No images found in directory: {image_dir}")
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image



# Image Preprocessing
image_transforms = transforms.Compose([
    transforms.Resize((640, 640)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])



In [30]:
from torchvision import transforms
image_transforms = transforms.Compose([
    transforms.Resize((640, 640)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Typical ImageNet normalization
])

In [31]:
# Create Dataset and Dataloader
calibration_dataset = CalibrationDataset(
    image_dir="C:/Lesson/EAI/Final/custom_yolov8/dataset/images/val/mix_all",
    transform=image_transforms
)

calibration_dataloader = DataLoader(
    calibration_dataset,
    batch_size=24,
    shuffle=True,
    num_workers=0  # Set to 0 to avoid multiprocessing issues
)

print(f"Calibration dataset contains {len(calibration_dataset)} images.")


Calibration dataset contains 1425 images.


In [32]:
from torch.ao.quantization import prepare

# Prepare the model for quantization
prepared_model = prepare(model)
print(prepared_model)

DetectionModel(
  (model): Sequential(
    (0): Conv(
      (conv): Conv2d(3, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(48, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (1): Conv(
      (conv): Conv2d(48, 96, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(96, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (2): C2f(
      (cv1): Conv(
        (conv): Conv2d(96, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(96, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (cv2): Conv(
        (conv): Conv2d(192, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(96, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
   

In [33]:
from tqdm import tqdm
# Run the model on the calibration dataset
for images in tqdm(calibration_dataloader):
    prepared_model(images)  # Forward pass for calibration

c:\Users\User\anaconda3\envs\myenv\lib\site-packages\PIL\Image.py:981: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


In [34]:
from torch.ao.quantization import convert

# Convert the model to quantized version
quantized_model = convert(prepared_model)

# Save the quantized model
torch.save(quantized_model.state_dict(), "quantized_yolov8s_model.pt")